# Лабораторная работа №3: Решающее дерево

1. Бейзлайн sklearn
2. Улучшение через подбор гиперпараметров
3. Собственная имплементация

## Импорт библиотек
Импортируем библиотеки для работы с решающими деревьями и метриками качества.

In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import f1_score, roc_auc_score, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import openml
import kagglehub
import os
import warnings
warnings.filterwarnings('ignore')
print("Импорт завершён")

Импорт завершён


### Датасет классификации: APS Failure at Scania Trucks
Загружаем данные, заполняем пропуски медианой и сэмплируем 15000 объектов. Стратифицированное разделение сохраняет баланс классов.

In [13]:
# Загрузка классификации
dataset = openml.datasets.get_dataset(41138)
X_clf, y_clf, _, _ = dataset.get_data(target=dataset.default_target_attribute)
y_clf_enc = (y_clf == 'pos').astype(int)
imputer = SimpleImputer(strategy='median')
X_clf_imp = pd.DataFrame(imputer.fit_transform(X_clf), columns=X_clf.columns)
np.random.seed(42)
idx = np.random.choice(len(X_clf_imp), 15000, replace=False)
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(
    X_clf_imp.iloc[idx], y_clf_enc.iloc[idx], test_size=0.2, random_state=42, stratify=y_clf_enc.iloc[idx]
)
print(f"Классификация: {X_clf_train.shape}")

Классификация: (12000, 170)


### Датасет регрессии: Avocado Prices
Загружаем данные о ценах авокадо. Кодируем категориальные признаки и разделяем на обучающую и тестовую выборки.

In [14]:
# Загрузка регрессии: Avocado Prices
path = kagglehub.dataset_download("neuromusic/avocado-prices")
df = pd.read_csv(os.path.join(path, "avocado.csv"))
df['type_enc'] = LabelEncoder().fit_transform(df['type'])
df['region_enc'] = LabelEncoder().fit_transform(df['region'])
features = ['Total Volume', '4046', '4225', '4770', 'Total Bags', 'year', 'type_enc', 'region_enc']
X_reg = df[features].values
y_reg = df['AveragePrice'].values
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
print(f"Регрессия: {X_reg_train.shape}")

Регрессия: (14599, 8)


## 2. Бейзлайн

Обучаем базовое дерево решений с ограничением глубины max_depth=5 для предотвращения переобучения. Деревья не требуют масштабирования признаков.

In [15]:
# Бейзлайн классификация
tree_clf_base = DecisionTreeClassifier(max_depth=5, random_state=42)
tree_clf_base.fit(X_clf_train, y_clf_train)
y_pred_base = tree_clf_base.predict(X_clf_test)
y_proba_base = tree_clf_base.predict_proba(X_clf_test)[:, 1]
f1_base = f1_score(y_clf_test, y_pred_base)
roc_base = roc_auc_score(y_clf_test, y_proba_base)
print(f"=== БЕЙЗЛАЙН: Decision Tree (max_depth=5) ===")
print(f"F1: {f1_base:.4f}, ROC-AUC: {roc_base:.4f}")

=== БЕЙЗЛАЙН: Decision Tree (max_depth=5) ===
F1: 0.6019, ROC-AUC: 0.8754


Обучаем базовое дерево регрессии с ограничением глубины max_depth=5. Оцениваем качество по RMSE и R².

In [16]:
# Бейзлайн регрессия
tree_reg_base = DecisionTreeRegressor(max_depth=5, random_state=42)
tree_reg_base.fit(X_reg_train, y_reg_train)
y_pred_reg_base = tree_reg_base.predict(X_reg_test)
rmse_base = np.sqrt(mean_squared_error(y_reg_test, y_pred_reg_base))
r2_base = r2_score(y_reg_test, y_pred_reg_base)
print(f"=== БЕЙЗЛАЙН: Decision Tree Regressor ===")
print(f"RMSE: {rmse_base:.4f}, R²: {r2_base:.4f}")

=== БЕЙЗЛАЙН: Decision Tree Regressor ===
RMSE: 0.2781, R²: 0.5185


## 3. Улучшение бейзлайна

Подбор гиперпараметров дерева решений методом GridSearchCV: максимальная глубина `max_depth` и минимальное число объектов в листе `min_samples_leaf`. Оптимизируем по F1-score.

In [17]:
# GridSearch классификация
param_grid = {'max_depth': [5, 10, 15, 20], 'min_samples_leaf': [1, 2, 5]}
grid_clf = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=3, scoring='f1', n_jobs=-1)
grid_clf.fit(X_clf_train, y_clf_train)
print(f"Лучшие параметры: {grid_clf.best_params_}")
y_pred_imp = grid_clf.predict(X_clf_test)
y_proba_imp = grid_clf.predict_proba(X_clf_test)[:, 1]
f1_imp = f1_score(y_clf_test, y_pred_imp)
roc_imp = roc_auc_score(y_clf_test, y_proba_imp)
print(f"=== УЛУЧШЕННЫЙ ===")
print(f"F1: {f1_imp:.4f} ({f1_imp-f1_base:+.4f})")
print(f"ROC-AUC: {roc_imp:.4f}")

Лучшие параметры: {'max_depth': 20, 'min_samples_leaf': 2}
=== УЛУЧШЕННЫЙ ===
F1: 0.5905 (-0.0115)
ROC-AUC: 0.8181


Подбор гиперпараметров для регрессии: глубина дерева и минимальное число объектов в листе. Увеличение глубины улучшает R², но может привести к переобучению.

In [18]:
# GridSearch регрессия
grid_reg = GridSearchCV(DecisionTreeRegressor(random_state=42), param_grid, cv=3, scoring='r2', n_jobs=-1)
grid_reg.fit(X_reg_train, y_reg_train)
print(f"Лучшие параметры: {grid_reg.best_params_}")
y_pred_reg_imp = grid_reg.predict(X_reg_test)
rmse_imp = np.sqrt(mean_squared_error(y_reg_test, y_pred_reg_imp))
r2_imp = r2_score(y_reg_test, y_pred_reg_imp)
print(f"=== УЛУЧШЕННЫЙ ===")
print(f"RMSE: {rmse_imp:.4f} ({rmse_imp-rmse_base:+.4f})")
print(f"R²: {r2_imp:.4f} ({r2_imp-r2_base:+.4f})")

Лучшие параметры: {'max_depth': 15, 'min_samples_leaf': 5}
=== УЛУЧШЕННЫЙ ===
RMSE: 0.2078 (-0.0703)
R²: 0.7311 (+0.2127)


## 4. Собственная имплементация

Реализуем алгоритм CART (Classification and Regression Trees) с нуля:
- Критерий Джини для классификации, MSE для регрессии
- Жадный перебор признаков и порогов для выбора лучшего разбиения
- Рекурсивное построение дерева с ограничением глубины
- Сэмплирование порогов для ускорения на больших данных

In [19]:
class MyDecisionTree:
    """Решающее дерево (CART)"""
    def __init__(self, max_depth=5, min_samples_leaf=1, task='classification'):
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.task = task
        self.tree = None
    
    def _gini(self, y):
        if len(y) == 0: return 0
        p = np.mean(y)
        return 2 * p * (1 - p)
    
    def _mse(self, y):
        return np.var(y) if len(y) > 0 else 0
    
    def _best_split(self, X, y):
        best_gain, best_feat, best_thresh = -np.inf, None, None
        criterion = self._gini if self.task == 'classification' else self._mse
        current = criterion(y)
        n = len(y)
        for feat in range(X.shape[1]):
            vals = np.unique(X[:, feat])
            if len(vals) > 20:
                vals = np.percentile(X[:, feat], np.linspace(5, 95, 15))
            for thresh in vals:
                left = X[:, feat] <= thresh
                n_l, n_r = np.sum(left), n - np.sum(left)
                if n_l < self.min_samples_leaf or n_r < self.min_samples_leaf:
                    continue
                gain = current - (n_l * criterion(y[left]) + n_r * criterion(y[~left])) / n
                if gain > best_gain:
                    best_gain, best_feat, best_thresh = gain, feat, thresh
        return best_feat, best_thresh
    
    def _build(self, X, y, depth):
        if depth >= self.max_depth or len(y) < 2 * self.min_samples_leaf or len(np.unique(y)) == 1:
            if self.task == 'classification':
                return {'leaf': True, 'value': int(np.mean(y) >= 0.5), 'prob': np.mean(y)}
            return {'leaf': True, 'value': np.mean(y)}
        feat, thresh = self._best_split(X, y)
        if feat is None:
            if self.task == 'classification':
                return {'leaf': True, 'value': int(np.mean(y) >= 0.5), 'prob': np.mean(y)}
            return {'leaf': True, 'value': np.mean(y)}
        left = X[:, feat] <= thresh
        return {'leaf': False, 'feature': feat, 'threshold': thresh,
                'left': self._build(X[left], y[left], depth + 1),
                'right': self._build(X[~left], y[~left], depth + 1)}
    
    def fit(self, X, y):
        self.tree = self._build(np.array(X), np.array(y), 0)
        return self
    
    def _pred_one(self, x, node):
        if node['leaf']: return node
        if x[node['feature']] <= node['threshold']:
            return self._pred_one(x, node['left'])
        return self._pred_one(x, node['right'])
    
    def predict(self, X):
        return np.array([self._pred_one(x, self.tree)['value'] for x in np.array(X)])
    
    def predict_proba(self, X):
        probs = np.array([self._pred_one(x, self.tree)['prob'] for x in np.array(X)])
        return np.c_[1 - probs, probs]

print("MyDecisionTree реализован!")

MyDecisionTree реализован!


Тестируем собственную реализацию дерева на задаче классификации. Результаты должны быть близки к sklearn при одинаковых параметрах.

In [20]:
# Своя классификация
my_tree_clf = MyDecisionTree(max_depth=5, task='classification')
my_tree_clf.fit(X_clf_train.values, y_clf_train.values)
my_pred = my_tree_clf.predict(X_clf_test.values)
my_proba = my_tree_clf.predict_proba(X_clf_test.values)[:, 1]
my_f1 = f1_score(y_clf_test, my_pred)
my_roc = roc_auc_score(y_clf_test, my_proba)
print(f"=== СВОЯ: Классификация ===")
print(f"F1: {my_f1:.4f} (sklearn: {f1_base:.4f})")
print(f"ROC-AUC: {my_roc:.4f}")

=== СВОЯ: Классификация ===
F1: 0.5789 (sklearn: 0.6019)
ROC-AUC: 0.8755


Тестируем собственную реализацию дерева на задаче регрессии. Критерий разбиения — MSE. Сравниваем с sklearn-бейзлайном.

In [21]:
# Своя регрессия
my_tree_reg = MyDecisionTree(max_depth=5, task='regression')
my_tree_reg.fit(X_reg_train, y_reg_train)
my_pred_reg = my_tree_reg.predict(X_reg_test)
my_rmse = np.sqrt(mean_squared_error(y_reg_test, my_pred_reg))
my_r2 = r2_score(y_reg_test, my_pred_reg)
print(f"=== СВОЯ: Регрессия ===")
print(f"RMSE: {my_rmse:.4f} (sklearn: {rmse_base:.4f})")
print(f"R²: {my_r2:.4f} (sklearn: {r2_base:.4f})")

=== СВОЯ: Регрессия ===
RMSE: 0.2779 (sklearn: 0.2781)
R²: 0.5194 (sklearn: 0.5185)


## Итоговая сводка

Сравниваем все модели: бейзлайн sklearn, улучшенный вариант с оптимальными гиперпараметрами и собственную реализацию CART.

In [22]:
print("="*70)
print("ИТОГОВАЯ СВОДКА: ЛАБОРАТОРНАЯ РАБОТА №3 (Decision Tree)")
print("="*70)
print(f"\n{'КЛАССИФИКАЦИЯ':-^70}")
print(f"{'Модель':<30} {'F1':<12} {'ROC-AUC':<12}")
print("-"*54)
print(f"{'Бейзлайн sklearn':<30} {f1_base:<12.4f} {roc_base:<12.4f}")
print(f"{'Улучшенный sklearn':<30} {f1_imp:<12.4f} {roc_imp:<12.4f}")
print(f"{'Своя реализация':<30} {my_f1:<12.4f} {my_roc:<12.4f}")
print(f"\n{'РЕГРЕССИЯ (Avocado Prices)':-^70}")
print(f"{'Модель':<30} {'RMSE':<12} {'R²':<12}")
print("-"*54)
print(f"{'Бейзлайн sklearn':<30} {rmse_base:<12.4f} {r2_base:<12.4f}")
print(f"{'Улучшенный sklearn':<30} {rmse_imp:<12.4f} {r2_imp:<12.4f}")
print(f"{'Своя реализация':<30} {my_rmse:<12.4f} {my_r2:<12.4f}")

ИТОГОВАЯ СВОДКА: ЛАБОРАТОРНАЯ РАБОТА №3 (Decision Tree)

----------------------------КЛАССИФИКАЦИЯ-----------------------------
Модель                         F1           ROC-AUC     
------------------------------------------------------
Бейзлайн sklearn               0.6019       0.8754      
Улучшенный sklearn             0.5905       0.8181      
Своя реализация                0.5789       0.8755      

----------------------РЕГРЕССИЯ (Avocado Prices)----------------------
Модель                         RMSE         R²          
------------------------------------------------------
Бейзлайн sklearn               0.2781       0.5185      
Улучшенный sklearn             0.2078       0.7311      
Своя реализация                0.2779       0.5194      
